<a href="https://colab.research.google.com/github/hideto7007/Object-Detection/blob/main/coco_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## coco_datast  
  
cocodataset作成手順  
1. images_data作成
2. annotation作成
3. jsonファイル作成  

※EfficientDetの場合、'iscrowd': 0をannotationファイル(dict)に追加する  
0は人間以外の領域を判別、1は人間の領域を判別


基本的な階層  
```
datasets/  
   -coco/  
        -train/  # 訓練画像
            -000000000001.jpg  
            -000000000002.jpg  
            -000000000003.jpg  
        -val/    # 検証画像
            -000000000004.jpg  
            -000000000005.jpg  
            -000000000006.jpg   
        -test/   # 評価画像
            -000000000004.jpg  
            -000000000005.jpg  
            -000000000006.jpg   
        -annotations  
            -instances_train.json  
            -instances_val.json 
            -instances_test.json

```  

※test_dataがある場合別で作成する

## train_coco_datatset作成

In [ ]:
# ライブラリのインストール
import json
import cv2
import os
import matplotlib.pyplot as plt
import collections

# JSONファイルの読み込み

ann_path = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/train_annotations/train_1014378_0057.json'

with open(ann_path) as f:
    train_annotation = json.load(f)
    
print(type(train_annotation))

# keyの一覧を取得する
print(train_annotation.keys())

# 「info」に含まれる値を出力
print('info:')
print(train_annotation['attributes'])
print('=====区切り=====')

# 「licenses」に含まれる値を出力
print('labels:')
print(train_annotation['labels'])

In [ ]:
# train_annotation['attributes']の値を変数train_annotationに代入する
train_images = train_annotation['attributes']

# train_imagesのデータ型を出力する
print(type(train_images))

# 「images」に含まれる値の個数を出力
print(len(train_images))

# 「images」に含まれる値を1つ出力
print('ある一枚の画像に関するメタ情報')
print(train_images)

In [ ]:
# train_annotation['attributes']の値を変数train_annotationに代入する
train_images = train_annotation['labels']

# train_imagesのデータ型を出力する
print(type(train_images))

# 「images」に含まれる値の個数を出力
print(len(train_images))

# 「images」に含まれる値を1つ出力
print('ある一枚の画像に関するメタ情報')
print(train_images[0])

In [ ]:
# 「annotations」に含まれる値を1つ出力
print('ある一つの物体に関するアノテーション情報: ')
print(train_annotation['attributes'])

# 「categories」データの出力
print('=====区切り=====')
print('labels: ')
print(train_annotation['labels'])

In [ ]:
dict1 = train_annotation['labels']

ca_name = [i.get('category') for i in dict1]
each_category = collections.Counter(ca_name)
each_category

In [ ]:
# categories確認

plt.figure(figsize=(20, 20))

# JSONファイルの読み込み
train_annotation = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/train_annotations/train_1014378_0057.json'
with open(train_annotation) as f:
    train = json.load(f)
    
# 画像ファイルのパスを生成する

image_path = train_annotation.replace('_annotations', '_images').replace('json', 'jpg')
image = cv2.imread(image_path)
# BGR -> RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# アノテーション(バウンディングボックス)情報を取得する
bbox_list = train['labels']

# カテゴリによって色を分ける
color_map = {
    '1_overall': (255, 0, 0), # 赤
    # '3_typography': (0, 0, 0), # 黒
    '4_illustration': (0, 0, 255), # 青
    # '5_stamp': (0, 255, 0), # 緑
    '7_caption': (255, 153, 50), # オレンジ
}

# 画像とバウンディングボックスを重ねて表示する
for bbox in bbox_list:
    
    # バウンディングボックスの位置情報を取得する
    x1 = bbox['box2d']['x1']
    y1 = bbox['box2d']['y1']
    x2 = bbox['box2d']['x2']
    y2 = bbox['box2d']['y2']
    
    # バウンディングボックスが囲む物体のカテゴリを取得する
    category = bbox['category']
    
    # カテゴリにしたがって色を決める
    color = color_map[category]
    
    # 物体を囲む矩形を描画する
    cv2.rectangle(img=image, 
                  pt1=(x1, y1), pt2=(x2, y2), 
                  color=color, thickness=2)
    
    # 物体の上に添えるテキストを描画する
    cv2.putText(img=image, text=category, org=(x1, y1 - 15), 
                fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, 
                color=color, thickness=2)

plt.imshow(image)
plt.show()

In [ ]:
# 各categories確認
categories = [
    {'id': 1, 'name': '1_overall'},
    # {'id': 2, 'name': '3_typography'},
    {'id': 3, 'name': '4_illustration'},
    # {'id': 4, 'name': '5_stamp'},
    {'id': 5, 'name': '7_caption'},
]
    
categories

In [ ]:
# imagesデータ作成

# 画像が格納されたディレクトリ名
image_dir = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/train_images/'

# # 全ての画像のファイル名をリストとして取得する
imfile_name_list = os.listdir(image_dir)
print(imfile_name_list)

# 画像IDの初期化
image_id = 1

# リストの初期化
images = []

# 各画像ごとの情報をひとつずつ取得
for imfile_name in imfile_name_list:
   
    # 画像のファイルパスを指定
    image_path = image_dir + imfile_name
    
    # 画像の読み込み
    image = cv2.imread(image_path)
    
    # 画像の高さ、幅を取得
    im_height, im_width, _ = image.shape

    # 画像1枚分の情報を辞書にまとめる
    image_dict = {
        'file_name': imfile_name,
        'height': im_height,
        'width': im_width,
        'id': image_id
    } 
    
    # 完成した画像1枚分の情報をimagesリストへ追加
    images.append(image_dict)
    
    # 画像IDを+1して次の画像へ
    image_id += 1
    
print(images)

In [ ]:
# annotationsデータ作成

# カテゴリ名をカテゴリIDにマッピングする辞書の定義
category_map = {
    '1_overall': 1,
    '2_handwritten': 2,
    '3_typography': 3,
    '4_illustration': 4,
    '5_stamp': 5, 
    '6_headline': 6,
    '7_caption': 7,
    '8_textline': 8,
    '9_table': 9,
}

image_dir = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/train_images/'
imfile_name_list = os.listdir(image_dir)
image_id = 1
images = []

# アノテーションデータが格納されたディレクトリ名
annot_dir = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/train_annotations/'

# 物体IDの初期化
annot_id = 1

# annotationsリストの初期化
annotations = []

for imfile_name in imfile_name_list:
   
    image_path = image_dir + imfile_name
    image = cv2.imread(image_path)
    im_height, im_width, _ = image.shape
    image_dict = {
        'file_name': imfile_name,
        'height': im_height,
        'width': im_width,
        'id': image_id
    } 
    images.append(image_dict)
    
    ### ここからannotationsリストに含める情報の取得へ ###
    # 画像と対応するアノテーションファイルのパスの指定
    annot_path = annot_dir + imfile_name.replace('jpg', 'json')
    
    # JSONファイルの読み込み
    with open(annot_path, encoding='utf-8') as f:
        annot = json.load(f)
    # 対象画像内の全物体に対するアノテーション情報リストの取得
    labels = annot['labels']
    
    # 各物体ごとのアノテーション情報をひとつずつ取得
    for label in labels:
        
        # バウンディングボックス位置情報の取得
        x1 = label['box2d']['x1']
        y1 = label['box2d']['y1']
        x2 = label['box2d']['x2']
        y2 = label['box2d']['y2']
        
        # bboxの高さ、幅の取得
        # (bboxの位置情報の表し方をXYXY形式 -> XYWH形式 に変更するため)
        box_width = x2 - x1
        box_height = y2 - y1
        
        # カテゴリIDの取得
        category = label['category']
        category_id = category_map[category]
                
        # 物体1つ分の情報を辞書にまとめる
        annot_dict = {
            'image_id': image_id,
            'bbox': [x1, y1, box_width, box_height],
            'category_id': category_id,
            'id': annot_id,
            'iscrowd': 0
        }
        
        # 完成した物体1つ分の辞書型データをannotationsリストへ追加
        annotations.append(annot_dict)
        
        # 物体IDを+1して次の物体へ
        annot_id += 1
        
    image_id += 1
    
print(annotations)

In [ ]:
# JSON(imageとannotationsデータ組み合わせ)ファイルを生成する

# categoriesリストの作成
categories = [
    {'id':1, 'name': '1_overall'},
    {'id':2, 'name': '2_handwritten'},
    {'id':3, 'name': '3_typography'},
    {'id':4, 'name': '4_illustration'},
    {'id':5, 'name': '5_stamp'},
    {'id':6, 'name': '6_headline'},
    {'id':7, 'name': '7_caption'},
    {'id':8, 'name': '8_textline'},
    {'id':9, 'name': '9_table'}
]

# カテゴリ名をカテゴリIDにマッピングする辞書の定義
category_map = {
    '1_overall': 1,
    '2_handwritten': 2,
    '3_typography': 3,
    '4_illustration': 4,
    '5_stamp': 5, 
    '6_headline': 6,
    '7_caption': 7,
    '8_textline': 8,
    '9_table': 9,
}

# 画像及びアノテーションデータが格納されたディレクトリ名の指定
image_dir = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/train_images/'
annot_dir = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/train_annotations/'

# 全ての画像のファイル名をリストとして取得
imfile_name_list = os.listdir(image_dir)

# 画像ID, 物体IDの初期化
image_id = 1
annot_id = 1

# リストの初期化
images = []
annotations = []

# 各画像ごとの情報をひとつずつ取得
for imfile_name in imfile_name_list:
   
    # 画像のファイルパスの指定
    image_path = image_dir + imfile_name
    # この画像と対応するアノテーションファイルのパスの指定
    annot_path = annot_dir + imfile_name.replace('jpg', 'json')
    
    # 画像の読み込み
    image = cv2.imread(image_path)
    
    # 画像の高さ、幅を取得
    im_height, im_width, _ = image.shape

    # 画像1枚分の情報を辞書にまとめる
    image_dict = {
        'file_name': imfile_name,
        'height': im_height,
        'width': im_width,
        'id': image_id
    } 
    
    # 完成した画像1枚分の情報をimagesリストへ追加
    images.append(image_dict)
    
    # JSONファイルを読み込み、
    # この画像内の全物体に対するアノテーション情報リスト取得
    with open(annot_path, encoding='utf-8') as f:
        annot = json.load(f)
    labels = annot['labels']
    
    # 各物体ごとのアノテーション情報をひとつずつ取得
    for label in labels:
        # bboxの位置情報の取得
        x1 = label['box2d']['x1']
        y1 = label['box2d']['y1']
        x2 = label['box2d']['x2']
        y2 = label['box2d']['y2']
        
        # bboxの高さ、幅の取得
        # (バウンディングボックスの位置情報の表し方をXYXY形式 -> XYWH形式 に変更するため)
        box_width = x2 - x1
        box_height = y2 - y1
        
        # カテゴリIDの取得
        category = label['category']
        category_id = category_map[category]
                
        # 物体1つ分のアノテーション情報を辞書にまとめる
        annot_dict = {
            'image_id': image_id,
            'bbox': [x1, y1, box_width, box_height],
            'category_id': category_id,
            'id': annot_id,
            'iscrowd': 0
        }
        
        # 完成した物体1つ分の情報をannotationsリストへ追加
        annotations.append(annot_dict)
        
        # 物体IDを+1して次の物体へ
        annot_id += 1
        
    # 画像IDを+1して次の画像へ
    image_id += 1

# COCO フォーマットと同様の構造を持つ辞書型データの作成
coco_format_dict = {
    'images': images,
    'annotations': annotations,
    'categories': categories,
    'iscrowd': 0
}

# 作成したCOCO フォーマットのアノテーションをJSONファイルに保存
with open('/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/annotations/instances_train_images.json', 'w') as f:
    json.dump(coco_format_dict, f)

# 作成したCOCO フォーマットのアノテーションデータを表示
print('--------------')
print('images')
print('--------------')
print(coco_format_dict['images'])

print('--------------')
print('annotations')
print('--------------')
print(coco_format_dict['annotations'])

print('--------------')
print('categories')
print('--------------')
print(coco_format_dict['categories'])

## test_coco_datatset作成

In [ ]:
test_ann_path = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/test_annotations/test_1016805_0080.json'

with open(test_ann_path) as f:
    test_annotation = json.load(f)
    
print(type(test_annotation))

# keyの一覧を取得する
print(test_annotation.keys())

In [ ]:
# 「info」に含まれる値を出力
print('info:')
print(test_annotation['attributes'])
print('=====区切り=====')

# # 「licenses」に含まれる値を出力
# print('labels:')
# print(test_annotation['labels'])

In [ ]:
# imagesデータ作成

# 画像が格納されたディレクトリ名
test_image_dir = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/test_images/'

# # 全ての画像のファイル名をリストとして取得する
imfile_name_list = os.listdir(test_image_dir)
print(imfile_name_list)

# 画像IDの初期化
image_id = 1

# リストの初期化
images = []

# 各画像ごとの情報をひとつずつ取得
for imfile_name in imfile_name_list:
   
    # 画像のファイルパスを指定
    image_path = test_image_dir + imfile_name
    
    # 画像の読み込み
    image = cv2.imread(image_path)
    
    # 画像の高さ、幅を取得
    im_height, im_width, _ = image.shape

    # 画像1枚分の情報を辞書にまとめる
    image_dict = {
        'file_name': imfile_name,
        'height': im_height,
        'width': im_width,
        'id': image_id
    } 
    
    # 完成した画像1枚分の情報をimagesリストへ追加
    images.append(image_dict)
    
    # 画像IDを+1して次の画像へ
    image_id += 1
    
print(images)

In [ ]:
# annotationsデータ作成

# カテゴリ名をカテゴリIDにマッピングする辞書の定義
# category_map = {
#     '1_overall': 1,
#     '2_handwritten': 2,
#     '3_typography': 3,
#     '4_illustration': 4,
#     '5_stamp': 5, 
#     '6_headline': 6,
#     '7_caption': 7,
#     '8_textline': 8,
#     '9_table': 9,
# }

image_dir = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/test_images/'
imfile_name_list = os.listdir(image_dir)
image_id = 1
images = []

# アノテーションデータが格納されたディレクトリ名
annot_dir = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/test_annotations/'

# 物体IDの初期化
annot_id = 1

# annotationsリストの初期化
annotations = []

for imfile_name in imfile_name_list:
   
    image_path = image_dir + imfile_name
    image = cv2.imread(image_path)
    im_height, im_width, _ = image.shape
    image_dict = {
        'file_name': imfile_name,
        'height': im_height,
        'width': im_width,
        'id': image_id
    } 
    images.append(image_dict)
    
    ### ここからannotationsリストに含める情報の取得へ ###
    # 画像と対応するアノテーションファイルのパスの指定
    annot_path = annot_dir + imfile_name.replace('jpg', 'json')
    
    # JSONファイルの読み込み
    with open(annot_path, encoding='utf-8') as f:
        annot = json.load(f)
    # # 対象画像内の全物体に対するアノテーション情報リストの取得
    # labels = annot['labels']
    
    # # 各物体ごとのアノテーション情報をひとつずつ取得
    # for label in labels:
        
    #     # バウンディングボックス位置情報の取得
    #     x1 = label['box2d']['x1']
    #     y1 = label['box2d']['y1']
    #     x2 = label['box2d']['x2']
    #     y2 = label['box2d']['y2']
        
    #     # bboxの高さ、幅の取得
    #     # (bboxの位置情報の表し方をXYXY形式 -> XYWH形式 に変更するため)
    #     box_width = x2 - x1
    #     box_height = y2 - y1
        
    #     # カテゴリIDの取得
    #     category = label['category']
    #     category_id = category_map[category]
                
        # 物体1つ分の情報を辞書にまとめる
        annot_dict = {
            'image_id': image_id,
            # 'bbox': [x1, y1, box_width, box_height],
            # 'category_id': category_id,
            'id': annot_id,
            'iscrowd': 0
        }
        
        # 完成した物体1つ分の辞書型データをannotationsリストへ追加
        annotations.append(annot_dict)
        
        # 物体IDを+1して次の物体へ
        annot_id += 1
        
    image_id += 1
    
print(annotations)

In [ ]:
# JSON(imageとannotationsデータ組み合わせ)ファイルを生成する

# categoriesリストの作成
# categories = [
#     {'id':1, 'name': '1_overall'},
#     {'id':2, 'name': '2_handwritten'},
#     {'id':3, 'name': '3_typography'},
#     {'id':4, 'name': '4_illustration'},
#     {'id':5, 'name': '5_stamp'},
#     {'id':6, 'name': '6_headline'},
#     {'id':7, 'name': '7_caption'},
#     {'id':8, 'name': '8_textline'},
#     {'id':9, 'name': '9_table'}
# ]

# カテゴリ名をカテゴリIDにマッピングする辞書の定義
# category_map = {
#     '1_overall': 1,
#     '2_handwritten': 2,
#     '3_typography': 3,
#     '4_illustration': 4,
#     '5_stamp': 5, 
#     '6_headline': 6,
#     '7_caption': 7,
#     '8_textline': 8,
#     '9_table': 9,
# }

# 画像及びアノテーションデータが格納されたディレクトリ名の指定
image_dir = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/test_images/'
annot_dir = '/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/test_annotations/'

# 全ての画像のファイル名をリストとして取得
imfile_name_list = os.listdir(image_dir)

# 画像ID, 物体IDの初期化
image_id = 1
annot_id = 1

# リストの初期化
images = []
annotations = []

# 各画像ごとの情報をひとつずつ取得
for imfile_name in imfile_name_list:
   
    # 画像のファイルパスの指定
    image_path = image_dir + imfile_name
    # この画像と対応するアノテーションファイルのパスの指定
    annot_path = annot_dir + imfile_name.replace('jpg', 'json')
    
    # 画像の読み込み
    image = cv2.imread(image_path)
    
    # 画像の高さ、幅を取得
    im_height, im_width, _ = image.shape

    # 画像1枚分の情報を辞書にまとめる
    image_dict = {
        'file_name': imfile_name,
        'height': im_height,
        'width': im_width,
        'id': image_id
    } 
    
    # 完成した画像1枚分の情報をimagesリストへ追加
    images.append(image_dict)
    
    # JSONファイルを読み込み、
    # この画像内の全物体に対するアノテーション情報リスト取得
    with open(annot_path, encoding='utf-8') as f:
        annot = json.load(f)
    # labels = annot['labels']
    
    # # 各物体ごとのアノテーション情報をひとつずつ取得
    # for label in labels:
    #     # bboxの位置情報の取得
    #     x1 = label['box2d']['x1']
    #     y1 = label['box2d']['y1']
    #     x2 = label['box2d']['x2']
    #     y2 = label['box2d']['y2']
        
    #     # bboxの高さ、幅の取得
    #     # (バウンディングボックスの位置情報の表し方をXYXY形式 -> XYWH形式 に変更するため)
    #     box_width = x2 - x1
    #     box_height = y2 - y1
        
        # # カテゴリIDの取得
        # category = label['category']
        # category_id = category_map[category]
                
        # 物体1つ分のアノテーション情報を辞書にまとめる
        annot_dict = {
            'image_id': image_id,
            # 'bbox': [x1, y1, box_width, box_height],
            # 'category_id': category_id,
            'id': annot_id,
            'iscrowd': 0
        }
        
        # 完成した物体1つ分の情報をannotationsリストへ追加
        annotations.append(annot_dict)
        
        # 物体IDを+1して次の物体へ
        annot_id += 1
        
    # 画像IDを+1して次の画像へ
    image_id += 1

# COCO フォーマットと同様の構造を持つ辞書型データの作成
coco_format_dict = {
    'images': images,
    'annotations': annotations,
    'iscrowd': 0
    # 'categories': categories
}

# 作成したCOCO フォーマットのアノテーションをJSONファイルに保存
with open('/content/drive/MyDrive/Yet-Another-EfficientDet-Pytorch-master/datasets/library_data/annotations/test_library.json', 'w') as f:
    json.dump(coco_format_dict, f)

# 作成したCOCO フォーマットのアノテーションデータを表示
print('--------------')
print('images')
print('--------------')
print(coco_format_dict['images'])

print('--------------')
print('annotations')
print('--------------')
print(coco_format_dict['annotations'])

# print('--------------')
# print('categories')
# print('--------------')
# print(coco_format_dict['categories'])